In [1]:
#Convert pdf to images
from pdf2image import convert_from_path

pdf_path = "periodico.pdf"
#Defining poppler_path
poppler_path = r"C:\poppler-24.08.0\Library\bin" 

try:
    images = convert_from_path(pdf_path, dpi=300, poppler_path=poppler_path)

    for i, image in enumerate(images):
        image.save(f"page_{i + 1}.jpg", "JPEG")

except Exception as e:
    print(f"An error occurred: {e}")

In [25]:
import ollama
import psutil
import GPUtil
import time
import os

def run_ai_query():
    start_time = time.time()

    import ollama

    response = ollama.chat(
        model='llama3.2-vision',
        messages=[{
            'role': 'user',
           'content': """Extract and return ONLY the text content from the image: verbatim, without translating, paraphrasing, or adding any explanations, notes, or formatting. The text is in Spanish, so preserve the original language and return ONLY the text in cohesive order, without any descriptions.
           ]
    """,
            'images': ['page_26.jpg']
        }]
    )

    print(response)


    execution_time = time.time() - start_time
    print("\n--- AI Query Execution Done ---")
    print(f"Execution Time: {execution_time:.4f} seconds")
    return response

def get_gpu_info():
    try:
        import GPUtil
        
        gpus = GPUtil.getGPUs()
        return [{
            "gpu_name": gpu.name,
            "gpu_memory_used": gpu.memoryUsed,  
            "gpu_memory_total": gpu.memoryTotal,  
            "gpu_load": gpu.load * 100,  
            "gpu_temperature": gpu.temperature
        } for gpu in gpus]
    except Exception as e:
        print(f"Error al obtener información de la GPU: {e}")
        return None

def get_system_info():
    return {
        "cpu_usage": psutil.cpu_percent(interval=1),  
        "memory_used": psutil.virtual_memory().used / (1024 ** 3),  
        "memory_total": psutil.virtual_memory().total / (1024 ** 3)
    }

def monitor_system():
    system_info = get_system_info()
    gpu_info = get_gpu_info()

    print("Sistema")
    print(f"CPU Usage: {system_info['cpu_usage']}%")
    print(f"Memory Used: {system_info['memory_used']:.2f} GB / {system_info['memory_total']:.2f} GB")

    if gpu_info:
        for i, gpu in enumerate(gpu_info):
            print(f"\n--- GPU {i+1} ({gpu['gpu_name']}) ---")
            print(f"GPU Load: {gpu['gpu_load']:.2f}%")
            print(f"GPU Memory Used: {gpu['gpu_memory_used']} MB / {gpu['gpu_memory_total']} MB")
    else:
        print("\nNo GPU detected.")


ai_response = run_ai_query()

monitor_system()

print("\n--- AI Response ---")
print(ai_response)


model='llama3.2-vision' created_at='2025-03-18T23:25:44.1618675Z' done=True done_reason='stop' total_duration=239219138900 load_duration=21125836800 prompt_eval_count=69 prompt_eval_duration=109633000000 eval_count=402 eval_duration=107634000000 message=Message(role='assistant', content='The text in the image is:\n\nAHUACHAPÁN\nCELEBRARÁN SUS FESTIVIDADES\nDespués de dos años de no realizar sus fiestas patronales debido a la pandemia del covid-19, el municipio de Turín celebrará sus festejos en honor al patrón San José. Darán inicio el sábado 12 de marzo con la elección y coronación de la reina de las festividades.\n\nUSULUTÁN\nAdvierten sobre modo de estafa\nAbordan a las víctimas en lugares públicos, les dan boletos para una supuesta rifA y les piden sus datos.\n\nCUSICÁN\nSINDICATO PIDE QUITAR A GERENTE MUNICIPAL\nDicen que no es apto para cargo. Gerente afirma se trata de asuntos personales.\n\nLA UNIÓN\nCAPTURADO POR INTENTAR ASESINAR A SU PAREJA\nFredy Yovany Hernández Maldonado 

In [24]:
message = ai_response.message.content
print(message)


The following text is extracted directly from the image:

[No text present]


In [12]:
start_char = "["
end_char = "]"

#Get AI response
message = ai_response.message.content

#Clean AI Response
start_index = message.find(start_char)
end_index = message.find(end_char)+1
json_message = message[start_index:end_index]

print(json_message)
#print(message)

[
  {
    "news title": "USULUTAN Advierten sobre modo de estafa",
    "content": "Abordan a las víctimas en lugares públicos, les dan boletos para una supuesta rifia y les piden sus datos."
  },
  {
    "news title": "CUCUTA Sindicato pide quitar a gerente municipal",
    "content": "Dicen que no es apto para cargo. Les afirma se trata de asuntos personales."
  },
  {
    "news title": "SAN SALVADOR 25 años estará en prisión por haber sido hallado culpable del delito de homicidio",
    "content": "Fue el año en que le quitó la vida a Nestor Andrés Rivera Parada, en Canton Tongolona, de Moncagua."
  }
]


In [13]:
import json

noticias = json.loads(json_message)
with open("noticias.json", "w", encoding="utf-8") as f:
    json.dump(noticias, f, ensure_ascii=False, indent=4)

print("El archivo JSON se ha guardado exitosamente.")

El archivo JSON se ha guardado exitosamente.
